In [ ]:
## 数据规整化：清理、转换、合并、重塑

In [1]:
# 1.合并数据集
# 1.1 merge：按照键进行行合并--数据路风格
# 1.2 concat： 沿着轴进行多对象堆叠
# 1.3 combine_first 可以将重复数据编接在一起，用一个对象的值填充另一个对象的缺失值
from pandas import Series, DataFrame  # Series 和 DataFrame使用次数很多，故直接引入本地命名空间
import pandas as pd
import numpy as np

In [6]:
# 1.1 merge
df1 = DataFrame({'key':['b','b','a','c','a','a','b'],
                'data1':range(7)})
df2 = DataFrame({'key':['a','b','d'],
                'data2':range(3)})
df3 = DataFrame({'key':['b','b','a','c','a'],
                'data3':range(5)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [4]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [5]:
pd.merge(df1,df2)  # 没有指定的话，merge就会将重叠列的列名当作键

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [7]:
pd.merge(df1,df3) # 我用这个证明，在默认条件下它是呈现一种笛卡尔乘积

,data1,key,data3
0,0,b,0
1,0,b,1
2,1,b,0
3,1,b,1
4,6,b,0
5,6,b,1
6,2,a,2
7,2,a,4
8,4,a,2
9,4,a,4


In [8]:
# 显式指定
pd.merge(df1, df2, on = 'key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [9]:
# 如果两个对象的列名不同，可以分别进行指定                    默认连接方式是inner--交集
df3 = DataFrame({'lkey':['b','b','a','c','a','a','b'],
                'data1':range(7)})
df4 = DataFrame({'rkey':['a','b','d'],
                'data2':range(3)})
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [10]:
# 方式还有left, right, outer
pd.merge(df1, df2, how = 'outer')  # outer是并集，inner是交集

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [11]:
pd.merge(df1, df2, how = 'left')

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [12]:
pd.merge(df1, df2, how = 'right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


In [13]:
# 两个键合并我们用left_on, right_on,但是如果多个键合并，我们可以采用直接传入由列名组成的列表
left = DataFrame({'key1':['foo', 'foo', 'bar'],
                 'key2':['one','two','one'],
                 'lval':[1, 2, 3]})
right = DataFrame({'key1':['foo','foo','bar','bar'],
                  'key2':['one','one','one','two'],
                  'rval':[4, 5, 6, 7]})
pd.merge(left, right, on = ['key1', 'key2'], how = 'outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [14]:
pd.merge(left, right, on = 'key1')  # 重复列名默认加后缀

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [15]:
pd.merge(left, right, on = 'key1', suffixes = ('_left', '_right'))  # suffixes参数设定后缀

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [16]:
# 有的时候连接键位于其索引中
# merge参数 --left_index : 将左侧行索引用作其连接键的列
#           --right_index: 将右侧行索引用作其连接键的列
left1 = DataFrame({'key':['a','b','a','a','b','c'],
                  'value':range(6)})
right1 = DataFrame({'group_val':[3.5, 7]}, index = ['a', 'b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [17]:
right1

,group_val
a,3.5
b,7.0


In [18]:
pd.merge(left1, right1, left_on = 'key', right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [19]:
pd.merge(left1, right1, left_on = 'key', right_index = True, how = 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [23]:
# 层次化索引
lefth = DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                  'key2':[2000, 2001, 2002, 2001, 2002],
                  'data':np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6, 2)),
                  index = [['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                           [2001, 2000, 2000, 2000, 2001, 2002]],
                          columns = ['event1','event2'])
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [24]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [25]:
pd.merge(lefth, righth, left_on = ['key1','key2'], right_index = True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [27]:
pd.merge(lefth, righth, left_on = ['key1','key2'],
        right_index = True, how = 'outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


In [28]:
# 即使同时使用合并双方的索引也没问题
# 也可以使用join实例方法，更为方便地实现按索引合并。
left2 = DataFrame([[1,2],[3,4],[5,6]],index = ['a','c','e'],
                 columns = ['Ohio','Nevada'])
right2 = DataFrame([[7,8],[9,10],[11,12],[13,14]],
                  index = ['b','c','d','e'], columns = ['Missiouri','Alabama'])
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [29]:
right2

,Missiouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [30]:
# 直接索引合并
left2.join(right2, how = 'outer')

,Ohio,Nevada,Missiouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [31]:
# 支持参数DataFrame的索引跟调用者DataFrame的某个列之间的连接
left1.join(right1, on = 'key')
# 也可以支持传入DataFrame

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [32]:
# 1.2 concat
# 用于numpy的合并原始NumPy数组的concatenation函数
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [34]:
np.concatenate([arr, arr], axis = 1)  # axis =1 是指列合并

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [35]:
s1 = Series([0, 1], index = ['a', 'b'])
s2 = Series([2, 3, 4], index = ['c','d','e'])
s3 = Series([5, 6], index = ['f', 'g'])
pd.concat([s1, s2, s3]) # 默认 axis = 0,直接黏贴

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [36]:
pd.concat([s1, s2, s3], axis = 1)  # axis = 1生成DataFrame

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [37]:
s4 = pd.concat([s1*5, s3])
pd.concat([s1, s4], axis = 1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [38]:
pd.concat([s1, s4], axis = 1, join = 'inner')

,0,1
a,0,0
b,1,5


In [39]:
# 使用join_axes指定要在其他轴上使用的索引
pd.concat([s1, s4], axis = 1, join_axes = [['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [40]:
# 在连接轴上创建一个层次化索引
result = pd.concat([s1, s2, s3], keys = ['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [41]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [43]:
# 如果沿着axis = 1合并， keys会成为DataFrame的开头
pd.concat([s1, s2, s3], axis = 1, keys = ['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [44]:
# 同样逻辑适用于DataFrame对象
# 传入的字典，字典的键会被当作keys选项的值
pd.concat({'level1':df1, 'level2':df2}, axis = 1)

level1     level2     
   data1 key  data2  key
0      0   b    0.0    a
1      1   b    1.0    b
2      2   a    2.0    d
3      3   c    NaN  NaN
4      4   a    NaN  NaN
5      5   a    NaN  NaN
6      6   b    NaN  NaN

In [45]:
# 此外还有两个用于管理层次化索引创建方式的参数
pd.concat([df1, df2], axis = 1, keys = ['level1', 'level2'],
         names = ['upper', 'lower'])

upper level1     level2     
lower  data1 key  data2  key
0          0   b    0.0    a
1          1   b    1.0    b
2          2   a    2.0    d
3          3   c    NaN  NaN
4          4   a    NaN  NaN
5          5   a    NaN  NaN
6          6   b    NaN  NaN

In [46]:
# ignore_index = True, 忽略与分析无关的DataFrame行索引

In [2]:
# 合并重叠数据 --数据打补丁
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
          index = ['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype = np.float64),
           index = ['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [3]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [4]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [5]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [7]:
b.combine_first(a)  # 用处同上：：打补丁

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [13]:
df1 = DataFrame({'a':[1, np.nan, 5, np.nan],
                'b':[np.nan, 2, np.nan, 6],
                'c':range(2, 18, 4)})
df2 = DataFrame({'a':[5, 4, np.nan, 3, 7],
                'b':[np.nan, 3, 4, 6, 8]})
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [17]:
# 2 重塑和轴向旋转
# 2.1 重塑层次化索引
# stack/unstack
#              --- stack:将数据的列“旋转”为行  ------默认操作的是最内层
#              --- unstack:将数据的行“旋转”为列
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index = pd.Index(['Ohio', 'Colorado'], name = 'state'),
                 columns = pd.Index(['one', 'two', 'three'], name = 'number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [18]:
result = data.stack()  # 将一个DataFrame转换为一个带层次化索引的Series
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [19]:
result.unstack()  # 将一个带层次化索引的Series转化为一个DataFrame

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [22]:
result.unstack(0)  # 外层

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [23]:
result.unstack('state')  # 将state行作为旋转的轴

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [24]:
# 如果不是所有级别值都能在各分组中找到的话，则unstack操作可能会引入缺失数据
s1 = Series([0, 1, 2, 3], index = ['a','b','c','d'])
s2 = Series([4, 5, 6], index = ['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys = ['one', 'two'])
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [25]:
data2.unstack().stack()  # stack默认操作会滤除缺失数据

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [26]:
data2.unstack().stack(dropna = False)  

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [ ]:
# 顿悟！ stack是堆叠的意思----对比柱形图，stacked模式

In [27]:
# 对DataFrame进行unstack操作，作为旋转轴的级别将会成为结果中的最低级别  
df = DataFrame({'left':result, 'right':result+5},
              columns = pd.Index(['left','right'], name = 'side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [30]:
df.unstack('number')

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [32]:
df.unstack('number').stack('side')

number          one  three  two
state    side                  
Ohio     left     0      2    1
         right    5      7    6
Colorado left     3      5    4
         right    8     10    9

In [28]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [29]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [34]:
alist = pd.read_csv(r'D:/Workshop/PyFile/pydata-book-master/ch07/macrodata.csv')

In [39]:
alist[:10]

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19
5,1960.0,2.0,2834.390,1792.9,298.152,460.400,1966.1,29.55,140.2,2.68,5.2,180.671,0.14,2.55
6,1960.0,3.0,2839.022,1785.8,296.375,474.676,1967.8,29.75,140.9,2.36,5.6,181.528,2.70,-0.34
7,1960.0,4.0,2802.616,1788.2,259.764,476.434,1966.6,29.84,141.1,2.29,6.3,182.287,1.21,1.08
8,1961.0,1.0,2819.264,1787.7,266.405,475.854,1984.5,29.81,142.1,2.37,6.8,182.992,-0.40,2.77
9,1961.0,2.0,2872.005,1814.3,286.246,480.328,2014.4,29.92,142.9,2.29,7.0,183.691,1.47,0.81


In [43]:
alist['quarter'][0] == 1
def quarter(x):
    return str(np.where(x == 1, '-03-31', np.where(x == 2, '-06-30', np.where(x == 3, '-09-30', '-12-31'))))
alist['monday'] = list(map(quarter, alist['quarter']))

In [54]:
str(alist['year'][0])+alist['monday'][0]

'1959.0-03-31'

In [2]:
# 3 数据转换
# 3.1 移除重复数据
data = DataFrame({'k1':['one']*3+['two']*4,
                 'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [3]:
# 通过duplicated方法返回一个布尔型Series，表示各行是否重复
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [4]:
# drop_duplicated方法返回一个移除了重复行的DataFrame
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [5]:
# 默认判断全部列。可以指定部分列进行重复项判断
data.drop_duplicates(['k1'])

,k1,k2
0,one,1
3,two,3


In [8]:
# 默认保留第一个出现的值的组合，传入take_last = True则保留最后一个
data.drop_duplicates(['k1'], keep = 'last')

,k1,k2
2,one,2
6,two,4


In [9]:
# 3.2 利用函数或映射进行数据转换
# 人话就是：map + 字典
# map是一种实现元素级转换以及其他数据清理工作的便捷方式
data = DataFrame({'food':['bacon','pulled pork','bacon','Pastrami','corned beef','Bacon','pastrami','honey ham','nova lox'],
                 'ounces':[4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [10]:
meat_to_animal = {
    'bacon':'pig',
    'pulled pork':'pig',
    'pastrami':'cow',
    'corned beef':'cow',
    'honey ham':'pig',
    'nova lox':'salmon'
}
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [11]:
# 另一种方式，将函数传入map中
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [13]:
# 3.3 替换值
# 实现替换值的三种方法： fillna--特例， map --映射， replace --更加简单、灵活
data = Series([1, -999, 2, -999, -1000, 3])
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [14]:
# 一次性替换多个值
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [15]:
# 对不同值替换不同值
data.replace([-999,-1000],[np.nan, 0])
# 或者用字典
data.replace({-999:np.nan,-1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [16]:
# 重命名轴索引
data = DataFrame(np.arange(12).reshape((3,4)),
                index = ['Ohio','Colorado','New York'],
                columns = ['one', 'two', 'three', 'four'])
data.index.map(str.upper)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

In [17]:
# 就地修改的方法 赋值
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [18]:
# 想要创建数据集的转换版， 而不是修改原始数据 ----rename
data.rename(index = str.title, columns = str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [19]:
# 结合字典型对象实现对部分周标签的更新
data.rename(index = {'OHIO':'INDIANA'},
           columns = {'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [20]:
# 使inplace = True，达成修改
_ = data.rename(index = {'OHIO':'INDIANA'}, inplace = True)

In [21]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [3]:
# 3.4 离散化和面元(bin)划分 -- cut函数
ages = [20,22,25,27,21,23,37,31,61,45,41,32]
bins = [18,25,35,60,100]  # 意图划分成，小于18，18-25，25-35，35-60，60-100几个组
cats= pd.cut(ages,bins)  # 可以传入right = False来修改为左闭右开
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [24]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [32]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [33]:
cats.value_counts()

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [34]:
cats.describe()

,counts,freqs
categories,,
"(18, 25]",5,0.416667
"(25, 35]",3,0.250000
"(35, 60]",3,0.250000
"(60, 100]",1,0.083333


In [4]:
# 设置面元名称 --就是bins的名称
group_names = ['Youth','YoungAdult','MiddleAged','Senior']
pd.cut(ages, bins, labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [5]:
# 如果向cut传入的是面元的数量而不是确切的面元边界，根据最小值和最大值计算等长面元
data = np.random.rand(20)
pd.cut(data, 4, precision = 4)  # precision是小数位数的参数

[(0.7438, 0.9832], (0.02451, 0.2649], (0.7438, 0.9832], (0.02451, 0.2649], (0.2649, 0.5043], ..., (0.7438, 0.9832], (0.02451, 0.2649], (0.02451, 0.2649], (0.2649, 0.5043], (0.7438, 0.9832]]
Length: 20
Categories (4, interval[float64]): [(0.02451, 0.2649] < (0.2649, 0.5043] < (0.5043, 0.7438] < (0.7438, 0.9832]]

In [6]:
# qcut是根据分位点进行分割，确保每个面元内含有的数据点相同
data = np.random.randn(1000)  # 正态分布
cats = pd.qcut(data, 4)  # 四分位点分
cats

[(0.707, 3.869], (-3.272, -0.687], (0.0249, 0.707], (0.0249, 0.707], (0.707, 3.869], ..., (-0.687, 0.0249], (-0.687, 0.0249], (0.0249, 0.707], (0.0249, 0.707], (-3.272, -0.687]]
Length: 1000
Categories (4, interval[float64]): [(-3.272, -0.687] < (-0.687, 0.0249] < (0.0249, 0.707] < (0.707, 3.869]]

In [7]:
pd.value_counts(cats)

(0.707, 3.869]      250
(0.0249, 0.707]     250
(-0.687, 0.0249]    250
(-3.272, -0.687]    250
dtype: int64

In [8]:
# 自定义分为点
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1])

[(1.267, 3.869], (-1.241, 0.0249], (0.0249, 1.267], (0.0249, 1.267], (0.0249, 1.267], ..., (-1.241, 0.0249], (-1.241, 0.0249], (0.0249, 1.267], (0.0249, 1.267], (-1.241, 0.0249]]
Length: 1000
Categories (4, interval[float64]): [(-3.272, -1.241] < (-1.241, 0.0249] < (0.0249, 1.267] < (1.267, 3.869]]

In [9]:
# 检测和过滤异常值
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [12]:
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [17]:
data[(np.abs(data) > 3).any(1)]  # axis = 1

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [18]:
# 排列和随机采样
# permutation -- 生成一个随机排列
df = DataFrame(np.arange(5*4).reshape(5,4))
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [19]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [20]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [22]:
df.ix[sampler]

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [23]:
df.loc[sampler]

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [24]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [25]:
# 非替换式采样  -- np.random.permutation
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


In [26]:
# 替换式采样  np.random.randint
bag = np.array([5,7,-1,6,4])
sampler = np.random.randint(0,len(bag),size = 10)
sampler

array([4, 4, 2, 2, 2, 0, 3, 0, 4, 1])

In [27]:
draws = bag.take(sampler)
draws

array([ 4,  4, -1, -1, -1,  5,  6,  5,  4,  7])

In [29]:
# 将分类变量转化为哑变量矩阵 -- get_dummies
df = DataFrame({'key':['b','b','a','c','a','b'],
               'data1':range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [30]:
# 哑变量矩阵加列名前缀，以便进行合并
dummies = pd.get_dummies(df['key'],prefix = 'key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [8]:
# 如果某行同属于多个分类
mnames = ['movie_id','title','genres']
movies = pd.read_table(r'D:/Workshop/PyFile/pydata-book-master/ch02/movielens/movies.dat', sep = '::', header = None, names = mnames, engine = 'python')
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [43]:
genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))  # 把原本迭代的部分变得可以带入
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns = genres)
for i, gen in enumerate(movies.genres):
    dummies.loc[i, gen.split('|')]=1
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.loc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

In [41]:
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [2]:
values = np.random.rand(10)
values

array([ 0.73827179,  0.23229929,  0.05105071,  0.52658268,  0.67282271,
        0.73327432,  0.64984665,  0.46650085,  0.07251129,  0.42126032])

In [6]:
# 妙计安天下
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins, right = False))  # get_dummies 是将分类变量转化成哑变量矩阵，pd.cut是将数据离散化

,"[0.0, 0.2)","[0.2, 0.4)","[0.4, 0.6)","[0.6, 0.8)","[0.8, 1.0)"
0,0,0,0,1,0
1,0,1,0,0,0
2,1,0,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0
5,0,0,0,1,0
6,0,0,0,1,0
7,0,0,1,0,0
8,1,0,0,0,0
9,0,0,1,0,0


In [10]:
### 正则表达式
# 1 连接与分割
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [11]:
pieces = [x.strip() for x in val.split(',')]  # 你看出来了，.strip()是去除了字符串内的空格
pieces

['a', 'b', 'guido']

In [12]:
a, b, c = pieces
a + '::' + b + '::' + c

'a::b::guido'

In [13]:
'::'.join(pieces)  # 一种更便捷高效的方法

'a::b::guido'

In [14]:
# 2 子串定位
'guido' in val

True

In [15]:
val.index(',')  # index找到会返回1，找不到返回异常

1

In [18]:
val.find(':')  # find找到返回1， 找不到返回-1

-1

In [22]:
val.count(',')  # count返回出现的次数

2

In [23]:
val.replace(',','::')  # 替换，常用于删除模式

'a::b:: guido'

In [25]:
val.replace(',','')  # 删除

'ab guido'

In [26]:
'asfdafsa'.startswith('a')

True

In [27]:
'fuckyou'.endswith('you')

True

In [28]:
# 其余方法
# rfind  返回匹配的最后一个字符串第一个字符在全文的位置
'fasjfajfjfsakjf'.rfind('a')

11

In [29]:
'fasjfajfjfsakjf'.find('a')

1

In [31]:
'boom'.upper()

'BOOM'

In [32]:
'NEW YORK'.lower()

'new york'

In [34]:
'fas'.ljust(5)

'fas  '

In [35]:
'fas'.rjust(5)

'  fas'

In [36]:
import re
text = 'foo  bar\t baz  \tqux'
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']